# Pré-processamento
1. Identificando e excluindo canais "ruins"
2. Identificando e excluindo trechos "ruins"
3. Removendo Slow drifts
4. Aplicando o notch filter (60Hz)
5. Reparando artefatos com ICA
6. Configurando uma referência para os dados de EEG
7. Salvar o arquivo pré-processado

In [ ]:
# Carregando pacotes
import os
import matplotlib
from copy import deepcopy
import numpy as np
import mne
from mne.preprocessing import ICA, corrmap, create_ecg_epochs, create_eog_epochs


In [ ]:
#Truque para "plotar" os gráficos em uma janela separada
matplotlib.use('Qt5Agg')

In [ ]:
# Abrindo o arquivo eeg
nome_arquivo = "eeg/raw/1/ME.edf"
raw = mne.io.read_raw_edf(nome_arquivo, preload=True)


In [ ]:
##### Removendo os canais que nao serao utilizados
ch_names = ['SystemTimestamp', 'Tilt X', 'Tilt Y', 'Tilt Z', 'ESUTimestamp', 'ECG']
raw.drop_channels(ch_names, on_missing='raise')

In [ ]:
##### Inserindo a montagem do sistema 10-20 no arquivo de eeg
easycap_montage = mne.channels.make_standard_montage("easycap-M1")
raw.set_montage(easycap_montage)

In [ ]:
## 1. Identificando e excluindo canais "ruins" 
# Clique no canal para marca-lo como ruim 
raw.plot()




In [ ]:
# vizualizando o canal
print(raw.info["bads"])

In [ ]:
# interpolate_bads() é usada para realizar a interpolação dos canais "ruins" ou "com defeito"
raw_interp = raw.copy().interpolate_bads(reset_bads=False)

raw.plot(butterfly=True, color="#00000022", bad_color="r") #plotar a interpolação

In [ ]:
######### 2. Identificando e excluindo trechos "ruins"

raw_interp.plot() #pressione A

In [ ]:
#Para vizualizar as  anotações

print(len(raw_interp.annotations))
print(set(raw_interp.annotations.onset))
print(set(raw_interp.annotations.duration))
print(set(raw_interp.annotations.description))


In [ ]:
# O parametro reject_by_annotation será utilizado a seguir na ICA

In [ ]:
# 3. Removendo Slow drifts
#Observe os locais de ruidos
raw_interp.compute_psd(fmax=100).plot(average=False, picks="data", exclude="reject_by_annotation")

In [ ]:
# Passando um filtro com a frquencia 0.1Hz
raw_fill = raw.copy().filter(l_freq=0.1, h_freq=None)

In [ ]:
#Observe que ruído de 60 Hz permanece
raw_fill.compute_psd(fmax=100).plot(average=False, picks="data", exclude="reject_by_annotation")

In [ ]:
# 4. Aplicando o notch filter (60Hz)
raw_notch = raw_fill.copy().notch_filter(freqs=60)

In [ ]:
#Observe que ruído de 60 Hz é atenuado
raw_notch.compute_psd(fmax=100).plot(average=False, picks="data", exclude="reject_by_annotation")

In [ ]:
# 5. Reparando artefatos com ICA
# Parâmetros
n_components = 9  
method = 'fastica' 
random_state = 23

In [ ]:
# Criando o objeto
ica = ICA(n_components=n_components, method=method, random_state=random_state)


In [ ]:
# "Filtrando" os dados com ICA
ica.fit(raw_notch, reject_by_annotation=True)

In [ ]:
# Formas de determinar a qualidade do ICA

#1. Variabilidade explicada

for i in range(9):
    explained_var_ratio = ica.get_explained_variance_ratio(raw_notch, components=[i])

    print('canal'+str(i)+' '+str(explained_var_ratio["eeg"]*100))

In [ ]:
# Formas de determinar a qualidade do ICA

#2. Fontes do ICA
ica.plot_sources(raw_notch, show_scrollbars=False)
print(raw_notch.info['ch_names']) # Coloquei aqui só para verificar os canais

In [ ]:
ica.plot_components()


In [ ]:
# Formas de determinar a qualidade do ICA

#3. Comparando as propriedades
ica.plot_properties(raw_notch, picks=[0,1])


In [ ]:
# Formas de determinar a qualidade do ICA

#4. Fazer a comparação excluído os dados

ica.plot_overlay(raw_notch, exclude=[0,1], picks='eeg')

In [ ]:
ica.exclude = [0,1] # Exclui os canais 0, 1

reconst_raw = raw_notch.copy()
ica.apply(reconst_raw)

reconst_raw.plot()


In [ ]:
# 6. Configurando uma referência para os dados de EEG
# Utilizando uma referência de canal (lóbulo da orelha esquerda)
# raw.set_eeg_reference(ref_channels=['A1'])

# Utilizando uma referencia  de canal (Cz)
# raw.set_eeg_reference(ref_channels=['Cz'])

# Utilizando a média dos canais mastóides como referência
# raw.set_eeg_reference(ref_channels=['M1', 'M2'])

# Utilizando uma se a referência bipolar (contralateral)
# raw.set_bipolar_reference(anode='[F3'], cathode=['F4'])



# Utilizando a média de todos os canais (Common average reference filter) ###
raw_avg_ref = reconst_raw.copy().set_eeg_reference(ref_channels="average")



In [ ]:
#Visualizar os dados 
raw_avg_ref.plot()

In [ ]:
# 7. Salvar o arquivo pré-processado
raw_avg_ref.save('eeg/preproc/1/ME.fif', overwrite=True)
